In [ ]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri='https://in03-a0e8499f9e3836d.api.gcp-us-west1.zillizcloud.com',
    token='af03123a9ccf876c1ab67438523167cb5d2226e2b3d4860ea6ab6114ecb7505741e500c31dc587b2b8e228e0917326c2f34df939', #  API key obtained from Zilliz Cloud
)

In [9]:
import os
from transformers import DistilBertTokenizer, DistilBertModel
from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType
import torch
from sentence_transformers import SentenceTransformer

# Initialize DistilBERT tokenizer and model
#tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#model = DistilBertModel.from_pretrained('distilbert-base-uncased')
#model = SentenceTransformer('sentence-transformers/sentence-t5-base')
#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

def read_and_clean_file(file_path):
    """Reads a file and cleans it of any unreadable characters"""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        return file.read()

def chunk_text(text, chunk_size=256):
    """Splits the text into chunks of specified size"""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

"""
def get_embedding(chunk):
    inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**inputs)
    return output.last_hidden_state.mean(dim=1).squeeze().tolist()
"""
def get_embedding(chunk):
    """Get the embedding vector for a chunk of text"""
    return model.encode(chunk)

def insert_docs_to_db(collection, batch_data):
    res = client.insert(
        collection_name=collection.name,  # Assuming collection has a 'name' attribute
        data=batch_data
    )
    return res

In [ ]:
import json
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Connect to cluster
connections.connect(
  alias='default', #  Public endpoint obtained from Zilliz Cloud
  uri='https://in03-a0e8499f9e3836d.api.gcp-us-west1.zillizcloud.com',
  secure=True,
  token='af03123a9ccf876c1ab67438523167cb5d2226e2b3d4860ea6ab6114ecb7505741e500c31dc587b2b8e228e0917326c2f34df939', #  API key obtained from Zilliz Cloud
)

In [3]:
# 2. Define fields
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="filename", dtype=DataType.VARCHAR, max_length=128),   
    FieldSchema(name="title_vector", dtype=DataType.FLOAT_VECTOR, dim=384),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2048),   
]

In [4]:
# 3. Build the schema
schema = CollectionSchema(
    fields,
    description="Schema of docs v2",
        enable_dynamic_field=False
)

In [5]:
# 4. Create collection
collection = Collection(
    name="python_docV2", 
    description="Python documentation collection v2",
    schema=schema
)

ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connect first.)>

In [ ]:
# 5. Create index

index_params = {
    "index_type": "AUTOINDEX",
    "metric_type": "L2",
    "params": {}
}

# To name the index, do as follows:
collection.create_index(
  field_name="title_vector", 
  index_params=index_params,
  index_name='title_vector_index'
)

In [ ]:
# Read files from the folder
folder_path = 'data/'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
total_files = len(all_files)

print(f"Starting processing of {total_files} files...")

for idx, filename in enumerate(all_files, 1):
    print(f"\nProcessing file {idx}/{total_files}: {filename}...")
    
    file_path = os.path.join(folder_path, filename)
    content = read_and_clean_file(file_path)
    chunks = chunk_text(content)
    total_chunks = len(chunks)
    
    print(f"Total chunks for {filename}: {total_chunks}")

    # Prepare batch data
    batch_data = []
    for chunk in chunks:
        vector = get_embedding(chunk)
        doc = {
            "filename": filename,
            "title_vector": vector
        }
        batch_data.append(doc)
    
    # Insert batch data to database
    insert_docs_to_db(collection, batch_data)

print("\nAll files processed!")

In [ ]:
prompt = get_embedding("Regular expression")
print(prompt)

In [ ]:
query_vector = get_embedding("how to use complex numbers")
res = client.search(
    collection_name="python_doc",
    data=[query_vector],
    output_fields=["filename"]
)

print(res)

In [ ]:
#print all the filenames of the response considering the structure is[[{'id': 443753998296271112, 'distance': 21.09891700744629, 'entity': {'filename': '3.txt'}}, {'id': 443753998296271115, 'distance': 21.15544891357422, 'entity': {'filename': '3.txt'}}
for i in res[0]:
    print(i['entity']['filename'])


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\loque\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import json
from nltk.tokenize import sent_tokenize
def get_sentences_from_text(text):
    """Return a list of sentences from the given text."""
    return sent_tokenize(text)

In [14]:
import os
import json

folder_path = 'data/'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
total_files = len(all_files)

print(f"Starting processing of {total_files} files...")

# This will be the resulting data to be saved to JSON
sentence_data = []
embedding_data = []
unique_id = 0  # Unique ID for sentences

for idx, filename in enumerate(all_files, 1):
    print(f"\nProcessing file {idx}/{total_files}: {filename}...")

    file_path = os.path.join(folder_path, filename)
    content = read_and_clean_file(file_path)
    sentences = get_sentences_from_text(content)

    for sentence in sentences:
        vector = get_embedding(sentence).tolist()  # Convert ndarray to a list

        # Append to sentence_data
        sentence_data.append({
            "id": unique_id,
            "filename": filename,
            "sentence": sentence
        })

        # Append to embedding_data
        embedding_data.append({
            "id": unique_id,
            "embedding": vector
        })

        unique_id += 1  # Increment unique_id for next sentence

# Saving to separate JSON files
with open("sentence_data.json", 'w', encoding='utf-8') as json_file:
    json.dump(sentence_data, json_file, ensure_ascii=False, indent=4)

with open("embedding_data.json", 'w', encoding='utf-8') as json_file:
    json.dump(embedding_data, json_file, ensure_ascii=False, indent=4)

print("\nAll files processed and results saved!")


FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'data/'

In [18]:
import os
import json
import re

def read_and_clean_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    return content

def preprocess_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove special characters
    sentence = re.sub(r'[^a-zA-Z0-9\s]', '', sentence)
    # Split into words and take only the first 256 words
    words = sentence.split()
    sentence = ' '.join(words[:256])
    return sentence


folder_path = '../data/'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
total_files = len(all_files)

print(f"Starting processing of {total_files} files...")

# This will be the resulting data to be saved to JSON
data = []

for idx, filename in enumerate(all_files, 1):
    print(f"\nProcessing file {idx}/{total_files}: {filename}...")

    file_path = os.path.join(folder_path, filename)
    content = read_and_clean_file(file_path)
    sentences = get_sentences_from_text(content)

    for sentence in sentences:
        sentence = preprocess_sentence(sentence)
        vector = get_embedding(sentence).tolist()  # Convert ndarray to a list
        doc = {
            "filename": filename,
            "sentence": sentence,
            "embedding": vector
        }
        data.append(doc)

# Saving to a JSON file
with open("processed_data.json", 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print("\nAll files processed and results saved to processed_data.json!")


Starting processing of 496 files...

Processing file 1/496: 1.txt...

Processing file 2/496: 10.txt...

Processing file 3/496: 100.txt...

Processing file 4/496: 101.txt...

Processing file 5/496: 102.txt...

Processing file 6/496: 103.txt...

Processing file 7/496: 104.txt...

Processing file 8/496: 105.txt...

Processing file 9/496: 106.txt...

Processing file 10/496: 107.txt...

Processing file 11/496: 108.txt...

Processing file 12/496: 109.txt...

Processing file 13/496: 11.txt...

Processing file 14/496: 110.txt...

Processing file 15/496: 111.txt...

Processing file 16/496: 112.txt...

Processing file 17/496: 113.txt...

Processing file 18/496: 114.txt...

Processing file 19/496: 115.txt...

Processing file 20/496: 116.txt...

Processing file 21/496: 117.txt...

Processing file 22/496: 118.txt...

Processing file 23/496: 119.txt...

Processing file 24/496: 12.txt...

Processing file 25/496: 120.txt...

Processing file 26/496: 121.txt...

Processing file 27/496: 122.txt...

Proce